# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [42]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,16.01,77,86,3.13,NZ,1738796831
1,1,araria,26.1500,87.5167,15.25,57,0,1.45,IN,1738796825
2,2,avarua,-21.2078,-159.7750,29.03,79,100,3.60,CK,1738796834
3,3,bethel,41.3712,-73.4140,-1.91,48,0,2.06,US,1738796837
4,4,nicoya,10.1483,-85.4520,28.99,56,66,4.30,CR,1738796683


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
ideal = city_data_df[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal = ideal.dropna()

# Display sample data
ideal.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,araria,26.1500,87.5167,15.25,57,0,1.45,IN,1738796825
3,3,bethel,41.3712,-73.4140,-1.91,48,0,2.06,US,1738796837
18,18,playa blanca,28.8643,-13.8281,17.06,73,0,7.94,ES,1738796861
21,21,prince rupert,54.3161,-130.3201,2.83,36,0,6.69,CA,1738796731
22,22,cabo san lucas,22.8909,-109.9124,26.17,33,0,5.14,MX,1738796686
31,31,kodiak,57.7900,-152.4072,1.73,75,0,0.00,US,1738796881
36,36,newman,37.3138,-121.0208,13.36,45,0,1.79,US,1738796887
45,45,pacific grove,36.6177,-121.9166,14.65,71,0,3.60,US,1738796654
49,49,anadyr,64.7500,177.4833,-36.67,65,0,4.00,RU,1738796909
52,52,el granada,37.5027,-122.4694,14.36,55,0,7.20,US,1738796916


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal.copy(deep=True)[["City", "Country", "Lat", "Lng", "Humidity"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
1,araria,IN,26.1500,87.5167,57,
3,bethel,US,41.3712,-73.4140,48,
18,playa blanca,ES,28.8643,-13.8281,73,
21,prince rupert,CA,54.3161,-130.3201,36,
22,cabo san lucas,MX,22.8909,-109.9124,33,
31,kodiak,US,57.7900,-152.4072,75,
36,newman,US,37.3138,-121.0208,45,
45,pacific grove,US,36.6177,-121.9166,71,
49,anadyr,RU,64.7500,177.4833,65,
52,el granada,US,37.5027,-122.4694,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [54]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    long = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(10)

Starting hotel search
araria - nearest hotel: SHALIMAR
bethel - nearest hotel: Hampton Inn Danbury
playa blanca - nearest hotel: H10 Lanzarote Princess
prince rupert - nearest hotel: Crest Hotel
cabo san lucas - nearest hotel: Comfort Rooms
kodiak - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
pacific grove - nearest hotel: Pacific Grove Inn
anadyr - nearest hotel: Гостевой дом
el granada - nearest hotel: Beach House
meadow lake - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
margaret river - nearest hotel: Margaret River Hotel
lihue - nearest hotel: Kauai Palms
kapa'a - nearest hotel: Pono Kai Resort
stanley - nearest hotel: Hotel 52
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
pitiquito - nearest hotel: Hotel Oasis
saipan - nearest hotel: Chalan Kanoa Beach Hotel
kassala - nearest hotel: No hotel found
nouadhibou - nearest hotel: Hotel valencia
port lincoln - nearest hotel: Boston Hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
1,araria,IN,26.1500,87.5167,57,SHALIMAR
3,bethel,US,41.3712,-73.4140,48,Hampton Inn Danbury
18,playa blanca,ES,28.8643,-13.8281,73,H10 Lanzarote Princess
21,prince rupert,CA,54.3161,-130.3201,36,Crest Hotel
22,cabo san lucas,MX,22.8909,-109.9124,33,Comfort Rooms
31,kodiak,US,57.7900,-152.4072,75,No hotel found
36,newman,US,37.3138,-121.0208,45,No hotel found
45,pacific grove,US,36.6177,-121.9166,71,Pacific Grove Inn
49,anadyr,RU,64.7500,177.4833,65,Гостевой дом
52,el granada,US,37.5027,-122.4694,55,Beach House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [68]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)